# REI505M Machine Learning - Final project
### Due: --------

**Names**: Axel Kristján Axelsson, Bjarni Haukur Bjarnason <br />
**Email**: aka30@hi.is, bhb23@hi.is

**1. Image classification**

s


In [29]:
from data_loader import MyDataLoader
%load_ext autoreload 
%autoreload 2
dl = MyDataLoader("flowers")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
dl.normalize_train_data()

In [24]:
dl.make_colorization_data()

In [30]:
try:
    del X
    del y

except:
    pass

X, y = dl.get_colorization_data()

TypeError: expected sequence object with len >= 0 or a single integer

In [7]:
from PIL import Image
import numpy as np

# print(X.shape)
# print(X_test.shape)
# im = Image.fromarray(X_train[0], mode="YCbCr")
# im.show()
# print(X_test[0])

(9262, 500, 500, 3)
(9262, 500, 500, 3)
[[[ 55  45  35]
  [ 56  46  36]
  [ 56  46  36]
  ...
  [ 47  43  31]
  [ 47  43  31]
  [ 47  43  31]]

 [[ 52  42  32]
  [ 53  43  33]
  [ 54  44  34]
  ...
  [ 52  48  36]
  [ 52  48  36]
  [ 52  48  36]]

 [[ 49  39  29]
  [ 50  40  30]
  [ 51  41  31]
  ...
  [ 55  51  39]
  [ 55  51  39]
  [ 55  51  39]]

 ...

 [[146 106 107]
  [146 106 107]
  [145 105 105]
  ...
  [165 158 166]
  [165 158 166]
  [165 158 166]]

 [[158 118 119]
  [158 118 119]
  [157 117 117]
  ...
  [175 168 176]
  [175 168 176]
  [175 168 176]]

 [[165 125 126]
  [164 124 125]
  [164 124 124]
  ...
  [180 173 181]
  [180 173 181]
  [180 173 181]]]


In [8]:
from PIL import Image
import numpy as np

img = Image.open("test.jpg").convert('L')
img = img.resize((500,500))
img.show()
arr = np.array(img)

In [18]:
from PIL import Image
import numpy as np

img = Image.open("bla.jpg")
img = img.resize((500,500))
x=np.ndarray((500,500,3), dtype=np.short)
img0 = Image.fromarray(x)

TypeError: Cannot handle this data type: (1, 1, 3), <i2